<a href="https://colab.research.google.com/github/shizoda/education/blob/main/agent/LangGraph(3)_Reflection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🦜🕸️ LangGraph (3): Reflection（自己省察）パターンの実装

これまでの回では、ReAct（思考と行動）や Human-in-the-loop（人間による修正）を扱いました。
今回は、エージェントが自身の出力を自分で評価し、自律的に修正する仕組みである Reflection（リフレクション）を実装します。

LLM は一度の生成（ゼロショット）で完全な回答を出すとは限りません。人間が推敲するように、AI にも下書き、見直し、修正のサイクルを行わせることで、最終的な出力品質を向上させることが可能です。

### 📚 学習目標
* **Generator & Reflector:** 作成者と批評家の役割分担の実装。
* **Cyclic Graph:** 品質向上のためのループ構造の設計。
* **Iterative Refinement:** 繰り返し回数による終了条件の制御。

In [ ]:
# ライブラリのインストール
!pip install -qU langgraph langchain-openai langchain-community termcolor grandalf

import os
from google.colab import userdata
from langchain_openai import ChatOpenAI
from termcolor import cprint

# APIキーの設定
try:
    os.environ["OPENROUTER_API_KEY"] = userdata.get("OPENROUTER_API_KEY")
    cprint("API Key loaded.", "green")
except Exception:
    cprint("Error: Please set OPENROUTER_API_KEY in secrets.", "red")

# LLMの初期化
# Reflection では創造と修正を行うため、temperature を少し上げて柔軟性を持たせます
llm = ChatOpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=os.environ["OPENROUTER_API_KEY"],
    model="deepseek/deepseek-chat-v3-0324",
    temperature=0.7
)

## 1. 🔍 Reflection の基本構造

Reflection パターンでは、主に2つの役割（ノード）を定義し、それらをループさせます。

1.  **Generator (作成者):** ユーザーの依頼に対する回答を作成、または批評を受けて修正版を作成します。
2.  **Reflector (批評家):** 作成された回答を読み、誤りや改善点を指摘します（回答そのものは書きません）。

### State の定義

今回はメッセージ履歴 `messages` を管理します。ループ回数は `messages` リストの長さをチェックすることで判定できるため、専用のカウンタ変数は必須ではありませんが、履歴を保持するために `add_messages` リデューサを使用します。

In [ ]:
from typing import Annotated, TypedDict, List
from langgraph.graph.message import add_messages
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage

class State(TypedDict):
    # add_messages リデューサにより、リストへの追加（append）が行われます
    messages: Annotated[List[BaseMessage], add_messages]

print("State schema defined.")

## 2. 📝 プロンプトとノードの実装

ここでは例として、短く魅力的な SNS 投稿を作成するエージェントを作成します。
Generator は投稿案を書き、Reflector は文字数、インパクト、ハッシュタグなどの観点で指摘を行います。

### 工夫点: Reflector の出力を HumanMessage にする
通常、AI の出力は `AIMessage` ですが、Reflection パターンでは Reflector の指摘をユーザー（人間）からの指摘として Generator に認識させると効果的です。そのため、Reflector ノードの戻り値を `HumanMessage` としてキャストする手法を使用します。

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

# --- Generator (作成者) ---
# 役割: ユーザーの要望と、(あれば)批評家の意見を踏まえて文章を書く
gen_prompt = ChatPromptTemplate.from_messages([
    ("system", "あなたはSNSのインフルエンサーです。ユーザーのトピックに基づき、魅力的で拡散されやすい短い投稿を作成してください。"),
    MessagesPlaceholder(variable_name="messages")
])
gen_chain = gen_prompt | llm

def generator_node(state: State):
    return {"messages": [gen_chain.invoke(state["messages"])]}


# --- Reflector (批評家) ---
# 役割: 直前の投稿案に対し、客観的かつ建設的なフィードバックを行う
reflect_prompt = ChatPromptTemplate.from_messages([
    ("system", """
    あなたは厳格なSNSコンサルタントです。直前の投稿案を批評し、改善点を列挙してください。
    以下の観点でチェックしてください：
    1. 140文字以内に収まっているか
    2. 読者の興味を惹くフックがあるか
    3. 適切なハッシュタグがあるか
    書き直した文章自体は提示せず、アドバイスだけを行ってください。
    """),
    MessagesPlaceholder(variable_name="messages")
])
reflect_chain = reflect_prompt | llm

def reflector_node(state: State):
    # 直前のメッセージ（Generatorの案）に対する批評を生成
    response = reflect_chain.invoke(state["messages"])

    # 【重要】Generator に人間からの指摘として受け取らせるため、HumanMessage に変換して返す
    # これにより、Generator は自分の回答、ユーザーからの指摘、修正案という文脈で推論できます
    return {"messages": [HumanMessage(content=response.content, name="Reflector")]}

## 3. 🔄 グラフの構築と終了条件

Generator と Reflector を交互に呼び出すループを作成します。
無限ループを防ぐため、一定回数繰り返したら終了する条件付きエッジ `should_continue` を定義します。

**フロー:**
START $\rightarrow$ Generator $\rightarrow$ (条件分岐: 終了 or Reflector) $\rightarrow$ Reflector $\rightarrow$ Generator ...

In [ ]:
from langgraph.graph import StateGraph, START, END

def should_continue(state: State):
    messages = state["messages"]

    # メッセージ数が 6 を超えたら終了
    # 構成: [User, Gen, Ref, Gen, Ref, Gen] = 6メッセージ
    # つまり3回書いても終了しない場合は強制終了とします
    if len(messages) > 6:
        return END
    return "reflect"

# グラフビルダーの初期化
builder = StateGraph(State)

# ノードの追加
builder.add_node("generate", generator_node)
builder.add_node("reflect", reflector_node)

# エッジの定義
builder.add_edge(START, "generate")

# 条件付きエッジ: generate の後に、終わるか reflect に行くか判断
builder.add_conditional_edges(
    "generate",
    should_continue,
    {"reflect": "reflect", END: END}
)

# reflect したら必ず generate に戻って修正させる
builder.add_edge("reflect", "generate")

graph = builder.compile()

# グラフ構造の可視化
from IPython.display import Image, display
try:
    display(Image(graph.get_graph().draw_mermaid_png()))
except Exception:
    pass

### 🚀 実行と確認

実際にトピックを与えて、どのように投稿が推敲されていくかを確認します。
`termcolor` を使用して、作成された案と批評を色分けして表示します。

In [ ]:
from termcolor import colored

topic = "朝のコーヒーの素晴らしさについて"

print(f"Topic: {topic}\n")

# グラフの実行
for event in graph.stream({"messages": [HumanMessage(content=topic)]}):
    for node, values in event.items():
        # 出力内容の取得
        message = values["messages"][-1]

        if node == "generate":
            print(colored(f"\n📝 **Generator (Draft):**", "cyan", attrs=["bold"]))
            print(message.content)
            print("-" * 40)
        elif node == "reflect":
            print(colored(f"\n🔍 **Reflector (Critique):**", "yellow", attrs=["bold"]))
            print(message.content)
            print("-" * 40)

### 💡 解説

出力結果を確認すると、以下のようなプロセスが見て取れます。

1.  **初稿:** まず一般的な投稿が作成されます。
2.  **批評:** Reflector が長すぎる、ハッシュタグが足りない、もっと感情に訴えてなどの指摘を行います。
3.  **修正:** Generator がその指摘を受け、修正版を作成します。

これを繰り返すことで、最終的な出力（最後の Generator の出力）は、初稿よりも品質の高いものになります。

## 4. ✍️ 【演習】 翻訳リファイナー（Translation Refiner）

Reflection パターンは翻訳の精度向上にも有効です。
直訳、不自然な点の指摘、自然な意訳というプロセスを構築します。

**課題:**
以下のコードの穴埋め `■■■` を完了させ、英語の技術文書を自然な日本語に翻訳するエージェントを作成してください。
ループは2回（初稿→指摘→修正稿）で終了するように設定します。

In [ ]:
# --- 1. ノードとプロンプトの定義 ---

# 翻訳者 (Translator) のプロンプト
trans_prompt = ChatPromptTemplate.from_messages([
    ("system", "あなたはプロの技術翻訳者です。ユーザーの英語テキストを日本語に翻訳してください。指摘がある場合は、それを反映して翻訳を修正してください。"),
    MessagesPlaceholder(variable_name="messages")
])
trans_chain = trans_prompt | llm

def translator_node(state: State):
    return {"messages": [trans_chain.invoke(state["messages"])]}

# レビュー担当 (Reviewer) のプロンプト
review_prompt = ChatPromptTemplate.from_messages([
    ("system", """
    あなたは日本語の技術文書レビュアーです。直前の翻訳を読み、以下の観点でフィードバックをしてください。
    ・日本語として不自然な表現はないか
    ・専門用語は正しく訳されているか（カタカナ語の乱用など）
    ・冗長な言い回しはないか
    翻訳自体は行わず、具体的な改善点のみを日本語で指摘してください。
    """),
    MessagesPlaceholder(variable_name="messages")
])
review_chain = review_prompt | llm

def reviewer_node(state: State):
    response = review_chain.invoke(state["messages"])
    # Generator に指摘として認識させるため、HumanMessage として返す
    return {"messages": [HumanMessage(content=response.content)]}

# --- 2. 終了条件の定義 ---
def stop_criterion(state: State):
    messages = state["messages"]
    # メッセージ数が 4 を超えたら終了するロジックを記述
    # 構成: [User, Trans, Review, Trans] = 4メッセージ
    if len(messages) >= ■■■:
        return END
    return "■■■"

# --- 3. グラフ構築 ---
trans_builder = StateGraph(State)

# ノードの追加
trans_builder.add_node("translator", ■■■)
trans_builder.add_node("reviewer", ■■■)

# エッジの定義
# 開始 -> 翻訳
trans_builder.add_edge(START, "translator")

# 条件付きエッジ: 翻訳 -> (終了判定) -> レビュー or 終了
trans_builder.add_conditional_edges(
    "translator",
    ■■■,
    {"reviewer": "reviewer", END: END}
)

# 循環エッジ: レビュー -> 翻訳 (再翻訳させる)
trans_builder.add_edge("■■■", "■■■")

trans_graph = trans_builder.compile()

# --- 4. 実行 ---
source_text = """
LangGraph is a library for building stateful, multi-actor applications with LLMs.
It extends LangChain to support cycles in the graph, which are essential for agentic behaviors.
"""

print("=== Translation Process ===")
for event in trans_graph.stream({"messages": [HumanMessage(content=source_text)]}):
    for node, values in event.items():
        content = values["messages"][-1].content
        if node == "translator":
             print(colored(f"\n[TRANSLATOR]\n{content}", "green"))
        else:
             print(colored(f"\n[REVIEWER]\n{content}", "blue"))

### 🏁 まとめ

今回は Reflection パターンを実装しました。
このパターンは、書いてから見直すという人間の基本的な作業フローを模倣したもので、複雑な推論や高品質な成果物が求められるタスクにおいて有効です。

次回は、複数のエージェントが協力してタスクを解決する Multi-Agent システムの構築に進みます。